# Artificial Neural Network

Artificial neural networks (ANNs) are a subset of machine learning that are modeled after the structure and function of the human brain. ANNs consist of layers of interconnected nodes that perform mathematical operations on input data to learn patterns and make predictions. They are widely used in various applications, such as image recognition, natural language processing, and predictive analytics.

### Importing the libraries

In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf    # tf.keras (bibliotheque deep learning)
import seaborn as sns

In [38]:
tf.__version__

'2.16.2'

## Part 1 - Data Preprocessing

### Importing the dataset

In [39]:
dataset = pd.read_csv('bank.csv')

In [40]:
dataset.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'deposit'],
      dtype='object')

In [41]:
dataset=dataset.drop_duplicates()

In [42]:
dataset.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [43]:
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [44]:
dataset['job'].value_counts().to_dict()

{'management': 2566,
 'blue-collar': 1944,
 'technician': 1823,
 'admin.': 1334,
 'services': 923,
 'retired': 778,
 'self-employed': 405,
 'student': 360,
 'unemployed': 357,
 'entrepreneur': 328,
 'housemaid': 274,
 'unknown': 70}

In [45]:
dataset['month'].value_counts().to_dict()

{'may': 2824,
 'aug': 1519,
 'jul': 1514,
 'jun': 1222,
 'nov': 943,
 'apr': 923,
 'feb': 776,
 'oct': 392,
 'jan': 344,
 'sep': 319,
 'mar': 276,
 'dec': 110}

In [46]:
dataset['default']=dataset['default'].map({'no':0,'yes':1})

In [47]:
dataset['marital']=dataset['marital'].map({'married':1,'single':0,'divorced':2})

In [48]:
dataset['education']=dataset['education'].map({'unknown':0,'primary':1,'secondary':2,'tertiary':3})

In [49]:
dataset['housing']=dataset['housing'].map({'no':0,'yes':1})

In [50]:
dataset['housing']

0        1
1        0
2        1
3        1
4        0
        ..
11157    1
11158    0
11159    0
11160    0
11161    0
Name: housing, Length: 11162, dtype: int64

In [51]:
dataset['loan']=dataset['loan'].map({'no':0,'yes':1})

In [52]:
dataset['contact']=dataset['contact'].map({'cellular':1,'unknown':0,'telephone':2})

In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['month'] = le.fit_transform(dataset['month'])

In [54]:
le = LabelEncoder()
dataset['job'] = le.fit_transform(dataset['job'])

In [55]:
dataset['poutcome'] = le.fit_transform(dataset['poutcome'])

In [56]:
dataset['deposit'] = le.fit_transform(dataset['deposit'])


In [57]:
dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,0,1,2,0,2343,1,0,0,5,8,1042,1,-1,0,3,1
1,56,0,1,2,0,45,0,0,0,5,8,1467,1,-1,0,3,1
2,41,9,1,2,0,1270,1,0,0,5,8,1389,1,-1,0,3,1
3,55,7,1,2,0,2476,1,0,0,5,8,579,1,-1,0,3,1
4,54,0,1,3,0,184,0,0,0,5,8,673,2,-1,0,3,1


In [58]:
dataset['pdays'].value_counts()

pdays
-1      8324
 92      106
 182      89
 91       84
 181      81
        ... 
 437       1
 728       1
 518       1
 828       1
 118       1
Name: count, Length: 472, dtype: int64

In [59]:
X=dataset.drop(columns="deposit").values

In [60]:
X

array([[ 59,   0,   1, ...,  -1,   0,   3],
       [ 56,   0,   1, ...,  -1,   0,   3],
       [ 41,   9,   1, ...,  -1,   0,   3],
       ...,
       [ 32,   9,   0, ...,  -1,   0,   3],
       [ 43,   9,   1, ..., 172,   5,   0],
       [ 34,   9,   1, ...,  -1,   0,   3]], dtype=int64)

In [61]:
y=dataset['deposit'].values

In [62]:
y

array([1, 1, 1, ..., 0, 0, 0])

### Splitting the dataset into the Training set and Test set

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state = 42)

### Feature Scaling

In [64]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [65]:
print(X_train)

[[ 2.9272381   0.16077033  1.92366589 ... -0.47447745 -0.366079
   0.51518369]
 [-0.18774699 -0.15047416  0.32394908 ...  1.27106587  0.08701793
  -1.49071487]
 [ 1.41183995  2.02823728  0.32394908 ... -0.47447745 -0.366079
   0.51518369]
 ...
 [-0.52450213  1.4057483   0.32394908 ... -0.47447745 -0.366079
   0.51518369]
 [ 0.82251844  0.16077033  0.32394908 ... -0.47447745 -0.366079
   0.51518369]
 [-0.94544606 -0.15047416 -1.27576772 ... -0.47447745 -0.366079
   0.51518369]]


## Part 2 - Building the ANN

### Initializing the ANN

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU

ann = Sequential()

ann.add(Dense(units=16))  
ann.add(LeakyReLU(alpha=0.015))

ann.add(Dense(units=16))
ann.add(LeakyReLU(alpha=0.015))

ann.add(Dense(units=16))
ann.add(LeakyReLU(alpha=0.015))

ann.add(Dense(units=16))
ann.add(LeakyReLU(alpha=0.015))

ann.add(Dense(units=16))
ann.add(LeakyReLU(alpha=0.015))

ann.add(Dense(units=1, activation='sigmoid'))  

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ann.fit(X_train, y_train, batch_size=32, epochs=120)

Epoch 1/120


c:\Users\CESAR\Desktop\gmc-course\.venv\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 620us/step - accuracy: 0.6135 - loss: 0.6431
Epoch 2/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - accuracy: 0.7887 - loss: 0.4628
Epoch 3/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.8021 - loss: 0.4399
Epoch 4/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.8066 - loss: 0.4326
Epoch 5/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 0.8088 - loss: 0.4222
Epoch 6/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - accuracy: 0.8190 - loss: 0.4161
Epoch 7/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.8226 - loss: 0.4032
Epoch 8/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - accuracy: 0.8183 - loss: 0.4087
Epoch 9/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.8285 - loss: 0.3936
Epoch 10/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.8308 - loss: 0.3941
Epoch 11/120
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.8155 - loss: 0.4102
Epoch 12/120
273/273 ━━━━━━━━━━

### Predicting the Test set results

In [67]:
y_pred = ann.predict(X_test) > 0.5

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step


### Making the Confusion Matrix

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1002  267]
 [ 215  972]]


0.8037459283387622

In [69]:
evaluation = ann.evaluate(X_test, y_test)

print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step - accuracy: 0.7867 - loss: 0.5285
Test Loss: 0.4752606451511383
Test Accuracy: 0.8037459254264832


In [70]:
from sklearn.metrics import classification_report
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred))

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step
              precision    recall  f1-score   support

           0       0.82      0.79      0.81      1269
           1       0.78      0.82      0.80      1187

    accuracy                           0.80      2456
   macro avg       0.80      0.80      0.80      2456
weighted avg       0.80      0.80      0.80      2456



Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.